In [22]:
import pandas as pd
import numpy as np
import sklearn.model_selection 
import sklearn.linear_model
import sklearn as sk
import plotly.graph_objects as go
from rpy2.robjects import r
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from sklearn.utils import resample

In [23]:
# reading auto dataset | declaring feature vectors and labels. 
dataset = pd.read_csv("./Auto.csv")
x = dataset.iloc[:,3].to_numpy() # horspower
y = dataset.iloc[:,0].to_numpy() # mpg

# deleting observations where x_i == ?  | parsing to float
indices_to_delete = [i for i,el in enumerate(x) if el == "?"] 
x = np.delete(x,indices_to_delete)
y = np.delete(y,indices_to_delete)
x = x.astype(np.float)
y = y.astype(np.float)


2.1. Realizar por completo la Actividad 1 pero ahora utilizando el lenguaje de programación Python.
Muestre imágenes de la respuesta en consola y redacte sus observaciones y conclusiones acerca de
cada ejercicio, así como de las gráficas obtenidas (colocar las imágenes en su reporte).

In [24]:
# splitting observations in half
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(x, y, test_size=0.5, random_state=1)

# polynomial lambda model -> (x^i)*coef_i + (x^i-1)*coef_i-1 + ... + bias
model = lambda x,coefs: sum([(x**i)*coef for i,coef in enumerate(reversed(coefs[0:-1]),start=1)])+coefs[-1]

# coeficient array and mse array 
coef_array = list()
mse_array = list()

for deg in range(1,4):
  # getting the coeficients for the model with a degree equal to 'i'
  coef_i = np.polyfit(x=x_train,y=y_train,deg=deg)
  # computing mse 
  predict_i = model(x_train,coef_i)
  mse_i = np.mean((y_train - predict_i)**2)
  # appending coef and mse
  coef_array.append(coef_i)
  mse_array.append(mse_i)

print(mse_array)

[23.172322563688436, 19.450442202363618, 19.444359249527423]


In [25]:
# splitting observations in half
x_train, x_test, y_train, y_test = sk.model_selection.train_test_split(x, y, test_size=0.5, random_state=2)

# polynomial lambda model -> (x^i)*coef_i + (x^i-1)*coef_i-1 + ... + bias
model = lambda x,coefs: sum([(x**i)*coef for i,coef in enumerate(reversed(coefs[0:-1]),start=1)])+coefs[-1]

# coeficient array and mse array 
coef_array = list()
mse_array = list()

for deg in range(1,4):
  # getting the coeficients for the model with a degree equal to 'i'
  coef_i = np.polyfit(x=x_train,y=y_train,deg=deg)
  # computing mse 
  predict_i = model(x_train,coef_i)
  mse_i = np.mean((y_train - predict_i)**2)
  # appending coef and mse
  coef_array.append(coef_i)
  mse_array.append(mse_i)

print(mse_array)

[24.663732413967836, 19.515744461302223, 19.5135796153207]


2.2. Realizar por completo la Actividad 2 pero ahora utilizando el lenguaje de programación Python.
Muestre imágenes de la respuesta en consola y redacte sus observaciones y conclusiones acerca de
cada ejercicio, así como de las gráficas obtenidas (colocar las imágenes en su reporte).

In [26]:
# fitting liner model for Auto
model = sk.linear_model.LinearRegression()
model.fit(x.reshape(-1,1),y)
print("INTERCEPT:",model.intercept_,"HORSEPOWER:",model.coef_)

INTERCEPT: 39.93586102117047 HORSEPOWER: [-0.15784473]


In [27]:
# KFolds tien
#folds = sk.model_selection.KFold(n_splits = len(y), shuffle = True, random_state = 100)
#folds = sk.model_selection.LeaveOneOut().get_n_splits(x)
#scores = sk.model_selection.cross_val_score(model, x.reshape(-1,1), y, cv=folds)
#print(scores)
boot = importr('boot')
r('''
Auto <- read.csv('./Auto.csv')
''')

r('''
glm.fit=glm(mpg~horsepower ,data=Auto)
cv.err =cv.glm(Auto ,glm.fit)
print(cv.err$delta)
''')
print('')

[1] 24.23151 24.23114



In [28]:
r('''
cv.error=rep (0,5)
for (i in 1:5){
 glm.fit=glm(mpg~poly(horsepower ,i),data=Auto)
 cv.error[i]=cv.glm (Auto ,glm.fit)$delta [1]
}
print(cv.error)
''')
print('')


[1] 24.23151 19.24821 19.33498 19.42443 19.03321



2.3. Realizar por completo la Actividad 3 pero ahora utilizando el lenguaje de programación Python.
Muestre imágenes de la respuesta en consola y redacte sus observaciones y conclusiones acerca de
cada ejercicio, así como de las gráficas obtenidas (colocar las imágenes en su reporte).

In [29]:
# create a KFold object with 10 splits 
folds = sk.model_selection.KFold(n_splits = 10, shuffle = True, random_state = 100)
scores = sk.model_selection.cross_val_score(model, x.reshape(-1,1), y, cv=folds)
print(scores)

[0.52614612 0.55012096 0.39495847 0.72501551 0.50305843 0.63314617
 0.58899073 0.68289297 0.53810027 0.65321772]


2.4. Realizar por completo la Actividad 4 pero ahora utilizando el lenguaje de programación Python.
Muestre imágenes de la respuesta en consola y redacte sus observaciones y conclusiones acerca de
cada ejercicio, así como de las gráficas obtenidas (colocar las imágenes en su reporte).

In [30]:
r('''
alpha.fn=function (data ,index){
 X=data$X [index]
 Y=data$Y [index]
 return ((var(Y)-cov (X,Y))/(var(X)+var(Y) -2* cov(X,Y)))
}
''')
print('')

In [31]:
#PortfolioDataset = pd.read_csv('./Portfolio.csv')
r('''
Portfolio <- read.csv('./Portfolio.csv')
''')
r('''
print(alpha.fn(Portfolio ,1:100))
''')
print('')

[1] 0.5758321



In [32]:
r(''' 
set.seed(1)
print(alpha.fn(Portfolio ,sample (100 ,100 , replace=T)))
''')
print('')

[1] 0.7368375



In [33]:
r(''' 
  print(boot(Portfolio ,alpha.fn,R=1000))
''')
print('')


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Portfolio, statistic = alpha.fn, R = 1000)


Bootstrap Statistics :
     original       bias    std. error
t1* 0.5758321 -0.001695873  0.09366347



In [34]:
r('''
Auto <- read.csv('./Auto.csv')
''')
r(''' 
  boot.fn=function(data ,index )
    return (coef(lm(mpg~horsepower ,data=data ,subset =index)))
  print(boot.fn(Auto ,1:392))
''')
print('')


(Intercept)  horsepower 
 39.9358610  -0.1578447 



In [35]:
r(''' 
  set.seed (1)
  print(boot.fn(Auto ,sample (392 ,392 , replace =T)))
  print(boot.fn(Auto ,sample (392 ,392 , replace =T)))
''')
print('')

(Intercept)  horsepower 
 40.3404517  -0.1634868 
(Intercept)  horsepower 
 40.1186906  -0.1577063 



In [36]:
r(''' 
  print(boot(Auto ,boot.fn ,1000))
''')
print('')


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
      original        bias    std. error
t1* 39.9358610  0.0544513229 0.841289790
t2* -0.1578447 -0.0006170901 0.007343073



In [37]:
r(''' 
  print(summary (lm(mpg~horsepower ,data=Auto))$coef)
''')
print('')

              Estimate  Std. Error   t value      Pr(>|t|)
(Intercept) 39.9358610 0.717498656  55.65984 1.220362e-187
horsepower  -0.1578447 0.006445501 -24.48914  7.031989e-81



In [38]:

r(''' 
  boot.fn=function (data ,index )
    coefficients(lm(mpg~horsepower +I( horsepower ^2) ,data=data , subset =index))
  set.seed (1)
  print(boot(Auto ,boot.fn ,1000))
  print(summary (lm(mpg~horsepower +I(horsepower ^2) ,data=Auto))$coef)
''')

print('')


ORDINARY NONPARAMETRIC BOOTSTRAP


Call:
boot(data = Auto, statistic = boot.fn, R = 1000)


Bootstrap Statistics :
        original        bias     std. error
t1* 56.900099702  3.511640e-02 2.0300222526
t2* -0.466189630 -7.080834e-04 0.0324241984
t3*  0.001230536  2.840324e-06 0.0001172164
                    Estimate   Std. Error   t value      Pr(>|t|)
(Intercept)     56.900099702 1.8004268063  31.60367 1.740911e-109
horsepower      -0.466189630 0.0311246171 -14.97816  2.289429e-40
I(horsepower^2)  0.001230536 0.0001220759  10.08009  2.196340e-21

